In [1]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import DistributionLoss
from sklearn.metrics import precision_score
import optuna
from sklearn.metrics import f1_score, precision_score, average_precision_score


In [2]:
#parameter tuning
#df=full preprocessed dataset

def tune_binary_model(df, horizon=30, n_trials=10,score_metric='f1'):

    df = df.copy()   
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = pd.to_datetime(df['date'])

    df['unique_id'] = 'main_series'
    df = df.rename(columns={'date': 'ds', 'payment_made': 'y'})

    min_val = df['payment'].min()
    max_val = df['payment'].max()
    df['payment'] = (df['payment'] - min_val) / (max_val - min_val) if max_val != min_val else 0
        
    print(f"Positive class ratio (y=1) in dataset: {df['y'].mean():.4f}")
    
    #creates future features using the test set
    def create_future_features(test_df):
        # Create future_df from test_df dates
        future_df = test_df[['ds']].copy()
        future_df['unique_id'] = 'main_series'
        
        # Merge with test_df features
        test_features = test_df[['ds', 'balance', 'day_of_week_sin', 'day_of_week_cos', 
                               'is_weekend', 'balance_changed']]
        
        future_df = future_df.merge(test_features, on='ds', how='left')
        
        return future_df[[
            'ds', 'unique_id', 'balance', 'balance_changed', 
            'day_of_week_sin', 'day_of_week_cos', 'is_weekend']]

    main_df = df[:-horizon].copy()
    train_df = main_df[:-horizon].copy()
    test_df = main_df[-horizon:].copy()
 
    future_exog_df = create_future_features(test_df)

    #define future and past feature columns
    hist_exog_vars = [
        'payment_count', 'days_since_last_payment', '7day_avg_payment', '30day_avg_payment',
        'cumulative_avg_zero_days', 'num_payments_from_start', 'payment',
        'day_of_week_sin', 'day_of_week_cos', 'is_weekend', 
        'balance', 'balance_changed']
    
    futr_exog_vars = ['day_of_week_sin', 'day_of_week_cos', 'is_weekend', 
                      'balance', 'balance_changed']


    def objective(trial):
        model = NBEATSx(
            h=horizon,
            input_size=trial.suggest_int("input_size", 80, 200),
            loss=DistributionLoss(distribution='Bernoulli', level=[80, 90]),
            dropout_prob_theta=trial.suggest_float("dropout_prob", 0.0, 0.3, step=0.05),
            scaler_type='robust',
            futr_exog_list=futr_exog_vars,
            hist_exog_list=hist_exog_vars,
            max_steps=trial.suggest_int("max_steps", 500, 1500),
            learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
            batch_size=trial.suggest_categorical("batch_size", [16, 32, 64]),
            #num_lr_decays=trial.suggest_int("num_lr_decays", 1, 5),
            val_check_steps=20,
            early_stop_patience_steps=5,
            random_seed=42 
        )
        nf = NeuralForecast(models=[model], freq='D')

        try:
            nf.fit(df=train_df, val_size=horizon)
            preds = nf.predict(futr_df=future_exog_df).reset_index(drop=False)
            preds = preds[['ds', 'NBEATSx-median']].rename(columns={'NBEATSx-median': 'payment_probability'})
            merged = preds.merge(test_df[['ds', 'y']], on='ds', how='left').dropna()
            y_true = merged['y']
            threshold = trial.suggest_float("threshold", 0.2, 0.8)
            y_pred = (merged['payment_probability'] > threshold).astype(int)

            # Choose metric
            if score_metric == 'f1':
                score = f1_score(y_true, y_pred, zero_division=0)
            elif score_metric == 'average_precision':
                score = average_precision_score(y_true, merged['payment_probability'])
            else:  # Default to precision
                score = precision_score(y_true, y_pred, zero_division=0)

            trial.set_user_attr("score", score)
            print(f"Trial {trial.number}: Score={score:.4f}, Threshold={threshold:.2f}, Params={trial.params}")
            return score
        except Exception as e:
            print(f"Trial failed: {e}")
            return 0.0
            
        

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Best parameters:", study.best_params)
    return study.best_params



In [5]:
#train and take predictions + uncertainity(of payment occurence)
#df=full dataset
#balance_pred_csv_path= reffers the predicted balance dataset (Isuru's predicted data)

def train_and_predict_binary_model(df, balance_pred_df, best_params, horizon=30):

    df = df.copy()  
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = pd.to_datetime(df['date'])

    df['unique_id'] = 'main_series'
    df = df.rename(columns={'date': 'ds', 'payment_made': 'y'})

    min_val = df['payment'].min()
    max_val = df['payment'].max()
    df['payment'] = (df['payment'] - min_val) / (max_val - min_val) if max_val != min_val else 0

    
    #create future features 
    def create_future_features(df, balance_pred_df):
        last_date = df['ds'].max()
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=horizon, freq='D')
        future_df = pd.DataFrame({'ds': future_dates})
        future_df['unique_id'] = 'main_series'

        balance_pred = balance_pred_df.copy()
        balance_pred['ds'] = pd.to_datetime(balance_pred['ds'])
        min_v = balance_pred['Forecast'].min()
        max_v = balance_pred['Forecast'].max()
        balance_pred['Forecast'] = (balance_pred['Forecast'] - min_v) / (max_v - min_v) if max_v != min_v else 0
        #balance_pred['date'] = pd.to_datetime(balance_pred['date']) 
        balance_pred = balance_pred.rename(columns={'Forecast': 'balance'})
        
        balance_pred = balance_pred[['ds', 'balance']]
        future_df = future_df.merge(balance_pred, on='ds', how='left')

        future_df['balance'] = future_df['balance'].fillna(0)

        future_df['day_of_week'] = future_df['ds'].dt.dayofweek
        future_df['day_of_week_sin'] = np.sin(2 * np.pi * future_df['day_of_week'] / 7)
        future_df['day_of_week_cos'] = np.cos(2 * np.pi * future_df['day_of_week'] / 7)
        future_df['is_weekend'] = future_df['day_of_week'].isin([5, 6]).astype(int)

        future_df = future_df.sort_values('ds').reset_index(drop=True)
        future_df['balance_changed'] = (future_df['balance'] != future_df['balance'].shift(1)).astype(int)
        future_df['balance_changed'] = future_df['balance_changed'].fillna(0)
        

        return future_df[[
            'ds', 'unique_id', 'balance', 'balance_changed',
            'day_of_week_sin', 'day_of_week_cos', 'is_weekend']]

    train_df = df[:-horizon].copy()
    ##train_df = df.copy()
    future_exog_df = create_future_features(train_df,balance_pred_df)

    #define future and past features columns
    #hist_exog_vars= historical features
    #futr_exog_vars=future features
    hist_exog_vars = [
        'payment_count', 'days_since_last_payment', '7day_avg_payment', '30day_avg_payment',
        'cumulative_avg_zero_days', 'num_payments_from_start', 'payment',
        'day_of_week_sin', 'day_of_week_cos', 'is_weekend', 
        'balance', 'balance_changed']
    
    futr_exog_vars = ['day_of_week_sin', 'day_of_week_cos', 'is_weekend', 
                      'balance', 'balance_changed']

    model = NBEATSx(
        h=horizon,
        input_size=best_params['input_size'],
        loss=DistributionLoss(distribution='Bernoulli', level=[80, 90]),
        dropout_prob_theta=best_params['dropout_prob'],
        scaler_type='robust',
        futr_exog_list=futr_exog_vars,
        hist_exog_list=hist_exog_vars,
        max_steps=best_params['max_steps'],
        learning_rate=best_params['learning_rate'],
        batch_size=best_params['batch_size'],
        #num_lr_decays=best_params['num_lr_decays'],
        val_check_steps=20,
        early_stop_patience_steps=5,
        random_seed=42 
    )

    nf = NeuralForecast(models=[model], freq='D')
    nf.fit(df=train_df, val_size=horizon)

    #take the occurence(1 or 0) and the probability(...%)
    # Predict probabilities
    preds = nf.predict(futr_df=future_exog_df).reset_index(drop=False)
    preds = preds[['ds', 'NBEATSx-median', 'NBEATSx']]
    preds.columns = ['ds', 'predicted_payment_occurence', 'predicted_occurence_probability']

    # Optional: Evaluate best threshold using last part of train_df
    val_df = df[-horizon:].copy()
    val_df = val_df[['ds', 'y']].rename(columns={'y': 'actual'})
    merged_val = preds.merge(val_df, on='ds', how='inner').dropna()
    
    def find_best_threshold(y_true, y_probs):
        thresholds = np.linspace(0.1, 0.9, 81)
        best_threshold = 0.5
        best_f1 = 0
        for t in thresholds:
            y_pred = (y_probs > t).astype(int)
            f1 = f1_score(y_true, y_pred)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = t
        return best_threshold, best_f1

    # Tune threshold using validation data
    best_threshold, best_f1 = find_best_threshold(
        y_true=merged_val['actual'], 
        y_probs=merged_val['predicted_occurence_probability']
    )
    print(f"[Threshold Tuning] Best threshold: {best_threshold:.2f}, F1: {best_f1:.4f}")

    # Apply optimal threshold
    preds['predicted_payment_occurence'] = (preds['predicted_occurence_probability'] > best_threshold).astype(int)

    # Uncertainty
    preds['uncertainty_percent'] = (1 - np.abs(preds['predicted_occurence_probability'] - 0.5) * 2) * 100

    
    return preds


In [4]:
if __name__ == "__main__":
    # Load data
    df = pd.read_excel('shihara_processed_data.xlsx')
    balance_df = pd.read_excel('forecast.xlsx')
    
    # Ensure dates are parsed
    df['date'] = pd.to_datetime(df['date'])

    # Convert Day (1-based) to datetime starting from 2025-01-02
    balance_df['ds'] = pd.to_datetime("2025-01-02") + pd.to_timedelta(balance_df['Day'] - 1, unit='D')
    
    # Run pipeline
    best_params = tune_binary_model(
        df=df, 
        horizon=30, 
        n_trials=10, 
        score_metric='average_precision'
    )
    
    predictions = train_and_predict_binary_model(
        df=df,
        balance_pred_df=balance_df,
        best_params=best_params,
        horizon=30
    )
    
    print(predictions.head())


[I 2025-07-12 09:55:21,484] A new study created in memory with name: no-name-26d48e96-d772-4316-b0de-0a904e2b19f4
Seed set to 42


Positive class ratio (y=1) in dataset: 0.2954


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params
--------------------------------------------------
0 | loss         | DistributionLoss | 5     
1 | padder_train | ConstantPad1d    | 0     
2 | scaler       | TemporalNorm     | 0     
3 | blocks       | ModuleList       | 5.0 M 
--------------------------------------------------
5.0 M     Trainable params
6.8 K     Non-trainable params
5.0 M     Total params
19.847    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-07-12 10:02:43,830] Trial 9 finished with value: 0.17333333333333334 and parameters: {'input_size': 184, 'dropout_prob': 0.1, 'max_steps': 1251, 'learning_rate': 0.00014524984505039364, 'batch_size': 16, 'threshold': 0.5783926872421646}. Best is trial 5 with value: 0.33333333333333337.


Trial 9: Score=0.1733, Threshold=0.58, Params={'input_size': 184, 'dropout_prob': 0.1, 'max_steps': 1251, 'learning_rate': 0.00014524984505039364, 'batch_size': 16, 'threshold': 0.5783926872421646}
Best parameters: {'input_size': 172, 'dropout_prob': 0.1, 'max_steps': 508, 'learning_rate': 0.00012029893236909243, 'batch_size': 16, 'threshold': 0.5545753259869556}


KeyError: 'Denormalized_Forecast'

In [6]:
predictions = train_and_predict_binary_model(
        df=df,
        balance_pred_df=balance_df,
        best_params=best_params,
        horizon=30
    )
    
print(predictions.head())

Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params
--------------------------------------------------
0 | loss         | DistributionLoss | 5     
1 | padder_train | ConstantPad1d    | 0     
2 | scaler       | TemporalNorm     | 0     
3 | blocks       | ModuleList       | 7.5 M 
--------------------------------------------------
7.5 M     Trainable params
12.3 K    Non-trainable params
7.5 M     Total params
30.119    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

[Threshold Tuning] Best threshold: 0.24, F1: 0.6957
          ds  predicted_payment_occurence  predicted_occurence_probability  \
0 2024-12-04                            1                            0.261   
1 2024-12-05                            0                            0.239   
2 2024-12-06                            0                            0.105   
3 2024-12-07                            0                            0.010   
4 2024-12-08                            0                            0.004   

   uncertainty_percent  
0            52.200001  
1            47.799999  
2            20.999998  
3             1.999998  
4             0.800002  


In [7]:
predictions.to_excel('classification_result_shiharadata.xlsx', index=False)
print("Saved predictions to classification_results_shihara.xlsx")

Saved predictions to classification_results_shihara.xlsx


In [8]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

# Get actuals from the last 30 days (horizon)
actuals_df = df.copy()
actuals_df['ds'] = pd.to_datetime(actuals_df['date']) if 'date' in actuals_df.columns else actuals_df['ds']
actuals_df = actuals_df.sort_values('ds')
actuals_df = actuals_df[['ds', 'payment_made']].rename(columns={'payment_made': 'actual_payment_occurence'})
actuals_df = actuals_df.tail(30)

# Merge with predictions
eval_df = predictions.merge(actuals_df, on='ds', how='inner')

# Extract ground truth and predictions
y_true = eval_df['actual_payment_occurence']
y_pred = eval_df['predicted_payment_occurence']
y_prob = eval_df['predicted_occurence_probability']

# Compute metrics
f1 = f1_score(y_true, y_pred, zero_division=0)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
accuracy = accuracy_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else 'Not Applicable (only one class present)'

# Print results
print("\n--- Model Evaluation Metrics ---")
print(f"F1 Score:       {f1:.4f}")
print(f"Precision:      {precision:.4f}")
print(f"Recall:         {recall:.4f}")
print(f"Accuracy:       {accuracy:.4f}")
print(f"ROC-AUC Score:  {roc_auc}")
print("--------------------------------")


--- Model Evaluation Metrics ---
F1 Score:       0.6957
Precision:      0.8000
Recall:         0.6154
Accuracy:       0.7667
ROC-AUC Score:  0.7714932126696834
--------------------------------


In [9]:
from sklearn.metrics import precision_recall_curve, roc_curve
import matplotlib.pyplot as plt

precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.title("Precision-Recall vs Threshold")
plt.show()


: 